In [1]:
import torch
from torch import optim
from torch import nn
from pytorch_pretrained_bert import TransfoXLModel, TransfoXLCorpus, TransfoXLTokenizer
import time, math, sys
sys.path.append('../')
import numpy as np

import models
from utils.data_loader import JSONFileDataLoader
from utils.transf_data_loader import JSONFileDataLoaderTransf
from utils.framework import FewShotREFramework, FewShotREModel
from models.sentence_encoding.basic_sentence_encoder import CNNSentenceEncoder
from models.proto_model import Proto

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
class TransformerSentenceEncoder(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        #self.device = "cpu"
        self.model = TransfoXLModel.from_pretrained('transfo-xl-wt103')
        self.model = self.model.to(self.device)
        #self.model.word_emb.requires_grad = False

    def forward(self, inputs):
        word = inputs.to(self.device)
        print(word.shape)
        l, m = self.model(word, None)
        print(l.shape, len(m))
        return l

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')
corpus = TransfoXLCorpus.from_pretrained('transfo-xl-wt103')

# va_iter = corpus.get_iterator('valid', 16, 128, device=device, ext_len=0)
te_iter = corpus.get_iterator('test', 8, 40, device=device, ext_len=0)

cuda


In [4]:
model_name = 'proto'
N = 5
K = 5
max_length = 40
train_data_loader = JSONFileDataLoaderTransf('../../FewRel/data/train.json',
                                             max_length=max_length, cuda=False)
val_data_loader = JSONFileDataLoaderTransf('../../FewRel/data/val.json',
                                           max_length=max_length, cuda=False)

Loading data file...
Finish loading
Elimiating case sensitive problem...
Finish eliminating
Finish building
Pre-processing data...
Finish pre-processing
Loading data file...
Finish loading
Elimiating case sensitive problem...
Finish eliminating
Finish building
Pre-processing data...
Finish pre-processing


In [5]:
framework = FewShotREFramework(train_data_loader, val_data_loader, None)
encoder = TransformerSentenceEncoder()
model = Proto(encoder)

In [6]:
# %%time
# sentence_encoder = CNNSentenceEncoder(train_data_loader.word_vec_mat, max_length)
# model = Proto(sentence_encoder)
# framework.train(model, model_name, 4, 20, N, K, 5, train_iter=20000)

In [7]:
# mems = None
# for idx, (data, target, seq_len) in enumerate(te_iter):
#     print(data.shape)
#     ret = encoder.model(data, mems)
#     loss, mems = ret
#     loss = loss.mean()
#     break

In [8]:
# from pytorch_openai_transformer_lm.model_pytorch import TransformerModel
# from pytorch_openai_transformer_lm.model_pytorch import load_openai_pretrained_model, DEFAULT_CONFIG

# args = DEFAULT_CONFIG
# model = TransformerModel(args)
# load_openai_pretrained_model(model)

In [9]:
# encoder.model.eval()
# total_len, total_loss = 0, 0.
# with torch.no_grad():
#     mems = None
#     for idx, (data, target, seq_len) in enumerate(te_iter):
#         ret = encoder.model(data, mems)
#         loss, mems = ret
#         loss = loss.mean()
#         total_loss += seq_len * loss.item()
#         total_len += seq_len
#         break
# print(total_loss / total_len)

In [10]:
# %%time
# support, query, label = train_data_loader.next_batch(2, N, K, 5)
# encoder.model.eval()
# with torch.no_grad():    
#     encoder(support)

In [13]:
%%time
support, query, label = train_data_loader.next_batch(1, 7, 1, 5)
encoder(support)

torch.Size([7, 40])
torch.Size([7, 40, 1024]) 18
CPU times: user 141 ms, sys: 180 ms, total: 321 ms
Wall time: 343 ms


In [59]:
support.keys()
query.keys()

dict_keys(['word', 'pos1', 'pos2', 'mask'])

In [61]:
label.shape

torch.Size([20, 25])

In [63]:
support['word'].shape

torch.Size([500, 40])